## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-25-10-49-29 +0000,nyt,Denmark Briefly Closes More Airports After Une...,https://www.nytimes.com/2025/09/25/world/europ...
1,2025-09-25-10-49-00 +0000,wsj,Denmark Says New Drone Flights Over Military B...,https://www.wsj.com/world/europe/denmark-says-...
2,2025-09-25-10-47-32 +0000,nyt,South Korea’s New Tattoo Law Legalizes the Pra...,https://www.nytimes.com/2025/09/25/world/asia/...
3,2025-09-25-10-45-24 +0000,bbc,'Anti-ICE' message on ammunition at Dallas sho...,https://www.bbc.com/news/articles/ckge9d26z1ro...
4,2025-09-25-10-43-00 +0000,wsj,Copper Surges as Freeport Mine Accident Clouds...,https://www.wsj.com/finance/commodities-future...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2347/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
108,trump,52
20,ice,15
7,new,15
264,president,12
24,shooting,12


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
183,2025-09-24-21-39-41 +0000,wapo,President Donald Trump wrote on TruthSocial th...,https://www.washingtonpost.com,97
298,2025-09-24-14-05-20 +0000,nypost,Trump threatens to sue ABC over Jimmy Kimmel’s...,https://nypost.com/2025/09/24/us-news/trump-th...,95
152,2025-09-24-22-55-47 +0000,nypost,Trump’s pro-Ukraine shift a ‘strategic move’ b...,https://nypost.com/2025/09/24/world-news/trump...,93
286,2025-09-24-15-09-41 +0000,cbc,Health Canada pushes back against Trump's clai...,https://www.cbc.ca/news/politics/health-canada...,83
170,2025-09-24-22-15-02 +0000,latimes,Trump's attacks on Kimmel and ABC put him at o...,https://www.latimes.com/politics/story/2025-09...,82


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
183,97,2025-09-24-21-39-41 +0000,wapo,President Donald Trump wrote on TruthSocial th...,https://www.washingtonpost.com
167,62,2025-09-24-22-20-12 +0000,nypost,"Vance tears into Democrats, calls out Gavin Ne...",https://nypost.com/2025/09/24/us-news/vance-te...
293,51,2025-09-24-14-37-59 +0000,nypost,Here’s what Jimmy Kimmel said — and if he apol...,https://nypost.com/2025/09/24/media/jimmy-kimm...
1,48,2025-09-25-10-49-00 +0000,wsj,Denmark Says New Drone Flights Over Military B...,https://www.wsj.com/world/europe/denmark-says-...
57,46,2025-09-25-08-08-40 +0000,wsj,Stock Market Today: Dow Futures Inch Up; White...,https://www.wsj.com/livecoverage/stock-market-...
96,44,2025-09-25-02-51-33 +0000,nypost,"Mom who lost daughter, 15, to suicide after on...",https://nypost.com/2025/09/24/world-news/mom-w...
287,39,2025-09-24-15-06-31 +0000,bbc,Israeli forces near Gaza City centre as Hamas ...,https://www.bbc.com/news/articles/c4gk9dreldxo...
102,34,2025-09-25-02-23-40 +0000,nypost,American Airlines accused of ‘running red ligh...,https://nypost.com/2025/09/24/us-news/american...
249,34,2025-09-24-18-22-17 +0000,bbc,Russia will expand aggression beyond Ukraine i...,https://www.bbc.com/news/articles/c5yg921rjrko...
141,33,2025-09-24-23-34-56 +0000,nypost,NYC GOP candidate Curtis Sliwa drops Guardian ...,https://nypost.com/2025/09/24/us-news/nyc-gop-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
